In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv("Data_Train.csv")

In [3]:
df.head()

,TITLE,RESTAURANT_ID,CUISINES,TIME,CITY,LOCALITY,RATING,VOTES,COST
0,CASUAL DINING,9438,"Malwani, Goan, North Indian","11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",Thane,Dombivali East,3.6,49 votes,1200
1,"CASUAL DINING,BAR",13198,"Asian, Modern Indian, Japanese",6pm – 11pm (Mon-Sun),Chennai,Ramapuram,4.2,30 votes,1500
2,CASUAL DINING,10915,"North Indian, Chinese, Biryani, Hyderabadi","11am – 3:30pm, 7pm – 11pm (Mon-Sun)",Chennai,Saligramam,3.8,221 votes,800
3,QUICK BITES,6346,"Tibetan, Chinese",11:30am – 1am (Mon-Sun),Mumbai,Bandra West,4.1,24 votes,800
4,DESSERT PARLOR,15387,Desserts,11am – 1am (Mon-Sun),Mumbai,Lower Parel,3.8,165 votes,300


In [8]:
df['VOTES'] = df['VOTES'].str.replace('votes','')

In [12]:
df.isna().sum()

TITLE               0
RESTAURANT_ID       0
CUISINES            0
TIME                0
CITY              112
LOCALITY           98
RATING              2
VOTES            1204
COST                0
dtype: int64

In [13]:
df.shape

(12690, 9)

In [16]:
df['VOTES'] = df['VOTES'].astype(float)

In [18]:
df['VOTES'].fillna(df['VOTES'].mean(),inplace=True)
df['VOTES'][0]

49.0

In [24]:
df['RATING'] = df['RATING'].str.replace('-','')

In [28]:
df['RATING'].value_counts().mean()

396.5

In [29]:
df['RATING'] = df['RATING'].str.replace('NEW','396.5')

In [39]:
df = df.dropna(axis=0,subset=['RATING'])

In [41]:
df['RATING'].value_counts()

3.9      1238
3.8      1190
4.0      1099
3.7      1086
3.6       951
4.1       936
3.5       771
4.2       723
396.5     707
3.4       575
4.3       556
          495
3.3       365
4.4       362
3.2       266
4.5       199
3.1       186
2.9       186
3.0       170
2.8       146
4.6       141
2.7        89
4.7        69
2.6        61
2.5        35
4.8        32
4.9        22
2.4        15
2.3        10
2.1         3
2.2         2
2.0         2
Name: RATING, dtype: int64

In [42]:
def convert_to_float(x):
    try:
        return float(x)
    except:
        return np.nan

In [43]:
df['RATING'] = df['RATING'].apply(lambda x: convert_to_float(x)) 

In [44]:
df['RATING'] = df['RATING'].astype(float)

In [57]:
df['LOCALITY'] = df['LOCALITY'].apply(lambda x : re.sub(r'[^A-Za-z]', '', str(x)).lower())

In [63]:
df['CITY'] = df['CITY'].apply(lambda x : re.sub(r'[^A-Za-z]', '', str(x)).lower())

In [68]:
df['CUISINES']=df['CUISINES'].apply( lambda x :" ".join(x.split(",")))

In [71]:
df['TITLE'] = df['TITLE'].apply(lambda x : re.sub(r'[^A-Za-z]', '', str(x)).lower())

In [73]:
df.isna().sum()

TITLE              0
RESTAURANT_ID      0
CUISINES           0
TIME               0
CITY               0
LOCALITY           0
RATING           495
VOTES              0
COST               0
dtype: int64

In [75]:
df['RATING'].fillna(df['RATING'].mean(),inplace=True)

In [77]:
cat_cols = ['TITLE','CUISINES','LOCALITY','CITY']

In [81]:
df_dummies = pd.get_dummies(data = df.copy(),columns=cat_cols)

In [82]:
df_dummies

,RESTAURANT_ID,TIME,RATING,VOTES,COST,TITLE_bakery,TITLE_bakerybeverageshop,TITLE_bakerycaf,TITLE_bakeryconfectionery,TITLE_bakerydessertparlor,...,CITY_velachery,CITY_veliaveetilhousevivekanandanagarelamakkara,CITY_vijayanagar,CITY_virarwest,CITY_vyttila,CITY_wagleestate,CITY_wardx,CITY_westmaredpally,CITY_westmumbai,CITY_whitefieldbangalore
0,9438,"11am – 4pm, 7:30pm – 11:30pm (Mon-Sun)",3.600000,49.000000,1200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,13198,6pm – 11pm (Mon-Sun),4.200000,30.000000,1500,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10915,"11am – 3:30pm, 7pm – 11pm (Mon-Sun)",3.800000,221.000000,800,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6346,11:30am – 1am (Mon-Sun),4.100000,24.000000,800,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15387,11am – 1am (Mon-Sun),3.800000,165.000000,300,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5618,12noon – 12:30AM (Mon-Sun),4.000000,550.000000,800,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,4383,"12noon – 11:30pm (Mon, Tue, Thu, Fri, Sat, Sun...",4.100000,509.000000,600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1249,1pm – 1am (Mon-Sun),4.300000,1612.000000,1600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,7062,7am – 10:30pm (Mon-Sun),3.900000,101.000000,300,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10109,"11am – 2pm, 4pm – 10:30pm (Mon-Sun)",3.300000,219.000000,200,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [89]:
X = df_dummies.drop(['RESTAURANT_ID','TIME','COST'],axis=1)
y = df_dummies['COST']

In [90]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [91]:
from sklearn.svm import SVR
model = SVR(kernel='rbf')
model.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [92]:
y_pred = model.predict(X_test)

In [93]:
def rmlse(y_test, y_pred):
    error = np.square(np.log10(y_pred +1) - np.log10(y_test +1)).mean() ** 0.5
    score = 1 - error
    return score

In [94]:
rmlse(y_test, y_pred)

0.7125487098250463

In [95]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor()
regr.fit(X_train,y_train)
y_predict_rf = regr.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [96]:
rmlse(y_test, y_predict_rf)

0.8166909978546022

In [97]:
import xgboost
xgb = xgboost.XGBRegressor()
xgb.fit(X_train,y_train)
y_predict_xgb = xgb.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[10:26:43] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [98]:
rmlse(y_test, y_predict_xgb)

0.7928584643033317

In [118]:
# Regression Example With Boston Dataset: Baseline
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.layers import Dropout

# define base model
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='sigmoid'))
NN_model.add(Dropout(0.1))
# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dropout(0.1))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='relu'))

In [1]:
# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['accuracy'])
NN_model.summary()

NameError: name 'NN_model' is not defined

In [2]:
NN_model.fit(X_train, y_train, epochs=100, batch_size=35, validation_split = 0.2)

NameError: name 'NN_model' is not defined

In [121]:
score = NN_model.evaluate(X_test, y_test, verbose=0) 
print('Test score:', score[0]) 
print('Test accuracy:', score[1])

Test score: 204.97926544150968
Test accuracy: 0.0025220680981874466
